In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']

# K Nearest Neighbors
*&copy; The Data Incubator*

In this notebook we'll explore the K Nearest Neighbors algorithm.  As the name implies, the prediction for a new point is based on the closest $k$ points in the training set (where closest is usually defined in terms of Euclidiean distance on the $p$-dimensional feature space).  In math terms, we're taking the Euclidean distance between the $p$-vectors $X_{j\cdot}$ as our metric.  As a classificaiton algorithm, it takes a majority vote of the nearest $k$ neighbors.  As a regression algorithm, it takes the average label of the nearest $k$ neighbors.

In [ ]:
# Load and display a sample of the iris dataset

from sklearn import neighbors, datasets
import pandas as pd
import numpy as np

# load the data
# The iris data set labels 3 flower types (y) by 4 different attributes of the flower (X).
# For more about the attributes, see http://mldata.org/repository/data/viewslug/datasets-uci-iris/
data = datasets.load_iris()
X = pd.DataFrame(data.data, columns=["sepal_length", "sepal_width", "petal_legnth", "petal_width"])
y = pd.Series(data.target)
X.head()

In [ ]:
# Process the nearest neighbors
nearest_neighbors = neighbors.NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nearest_neighbors.fit(X)
distances, indices = nearest_neighbors.kneighbors(X)

# Column $k$ gives the distances between each point and its $k$-th nearest neighbor from 0 to 4
# Column 0 is always itself.
# Each row is a separate point (note the first column)
print "Distance"
print pd.DataFrame(distances).head()
print "Indices"
print pd.DataFrame(indices).head()

In [ ]:
# We illustrate the effect of the number of neigbhors used on accruacy.

from sklearn import neighbors, cross_validation, grid_search
import matplotlib.pylab as plt

cv = cross_validation.ShuffleSplit(len(y), n_iter=20, test_size=0.2, random_state=42)
param_grid = { "n_neighbors": range(4, 24, 4) }
nearest_neighbors_cv = grid_search.GridSearchCV(neighbors.KNeighborsClassifier(), 
                                                param_grid=param_grid, cv=cv, 
                                                scoring='accuracy')

nearest_neighbors_cv.fit(X, y)
cv_accuracy = pd.DataFrame.from_records(
    [(score.parameters['n_neighbors'],
      score.mean_validation_score)
     for score in nearest_neighbors_cv.grid_scores_],
columns=['n_neighbors', 'accuracy'])

plt.plot(cv_accuracy.n_neighbors, cv_accuracy.accuracy)
plt.xlabel('n_neighbors')
plt.ylabel('accuracy')
plt.show()

**Question:**

1. Is this algorithm parametric on non-parametric?
1. As `n_neighbors` gets larger, the model becomes more accurate and then less.  What's happening?
1. For the classification version, what happens when you have one class being over represented?
1. The notion of distance in nearest neighbors is Euclidan in feature space.  For what kinds of input data might this be a problem?

## Normalizing Data.

Based on your answer to the last question, we will want to scale the individual features so that they have similar variance.  For each feature, we are going to subtract the mean and divide by the standard deviation:

$$ X_{ji}' = \frac{X_{ji} - \mu_i}{\sigma_i} $$

where $\mu_i$ is the mean of the $i$-th column (or feature) and $\sigma_i$ is the standard deviation.

**Question**: For $k$-Nearest-Neighbors, was subtracting the mean necessary?  For what other algorithms might subtracting the mean help?

**Exercise**: Let's put all each feature of the dataset on the same scale by normalizing.

In [ ]:
from sklearn import preprocessing

# first, let's see what the standard deviations are.
scaler = preprocessing.StandardScaler(copy=True).fit(X)
pd.DataFrame({
    "Mean": scaler.mean_,
    "Std": scaler.std_
}, index=X.columns)

**Exercise**: It looks like some of the features are on a larger numerical scale.  Let's put all the data on the same scale.

In [ ]:
X_copy = X.copy()  # Feature (bug?): transform modifies the original 
X_scaled = scaler.transform(X_copy)
pd.DataFrame({
    "Mean": X_scaled.mean(axis=0), 
    "Std": X_scaled.std(axis=0)
})

In [ ]:
# We are going to first scale and then apply KNeighbors.
# Pipeline allows us to chain together multiple transformers and then a regressor or classifier

from sklearn.pipeline import Pipeline

scaled_nearest_neighbors = Pipeline([('scaling', preprocessing.StandardScaler(copy=True)), 
                                     ('neighbors', neighbors.KNeighborsClassifier())])

param_grid = { "neighbors__n_neighbors": range(4, 24, 4) }    # parameters to Pipeline take the form [label]__[estimator_param]
scaled_nearest_neighbors_cv = grid_search.GridSearchCV(scaled_nearest_neighbors, 
                                                       param_grid=param_grid, cv=cv, 
                                                       scoring='accuracy')

scaled_nearest_neighbors_cv.fit(X, y)
scaled_cv_accuracy = pd.DataFrame.from_records(
    [(score.parameters['neighbors__n_neighbors'],
      score.mean_validation_score)
     for score in scaled_nearest_neighbors_cv.grid_scores_],
columns=['n_neighbors', 'accuracy'])

plt.plot(scaled_cv_accuracy.n_neighbors, scaled_cv_accuracy.accuracy, label="scaled_cv_accuracy")
plt.plot(cv_accuracy.n_neighbors, cv_accuracy.accuracy, label="cv_accuracy")
plt.xlabel('n_neighbors')
plt.ylabel('accuracy')
plt.legend(loc='lower center')
plt.show()

## Additional Resources

1. [Wikipedia](http://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)
1. The Scikit Learn package implements [K Nearest Neighbors](http://scikit-learn.org/stable/modules/neighbors.html).  It has both a [`KNeighborsClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) and [`KNeighborsRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

#### Exit Tickets
1. In KNN, does most computation take place during training or during testing?
1. After training, what must the model retain in memory in order to make predictions?
1. What are algorithms like KD-Ball and KD-Tree meant to do? And what's their drawback?

# Spoilers

# Answers

1. The algorithm is non-parametric.
1. As `n_neighbors` gets larger, we're increasing variance and decreasing bias so we expect to see greater and than less accuracy.
1. For the classification version, if one class is over-represented, then it will always win a majority vote.  Using an average of the nearest $k$ neighbors as a predictor may be useulf.
1. The notion of distance in nearest neighbors is Euclidan in feature space.  This is really bad if the features have very different scales.

1. It is probably good to normalize the data by dividing all the features by their standard deviation.  Note that subtracting a mean is not necessary.

*Copyright &copy; 2015 The Data Incubator.  All rights reserved.*